# | Practice 3-1 | Lightweight CNN

**Tensorflow Functional API**를 이용하여 이미지를 구분하는 MobileNet V2를 구현한다. <br>
그 중에서도 가장 핵심이라고 할 수 있는 **Linear bottleneck**과 **Inverted residuals**를 중점적으로 다룰 것이다.

---

## (Optional) Colab Setup

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
"""
Change directory to where this file is located
"""
%cd '/content/drive/MyDrive/Colab Notebooks/Ambient AI/Lesson_Homework_3'

/content/drive/MyDrive/Colab Notebooks/Ambient AI/Lesson_Homework_3


## Import Modules

In [24]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Add, ReLU, Input, Dense, Activation, Flatten, Conv2D, \
    DepthwiseConv2D, BatchNormalization, GlobalAveragePooling2D, Dropout
from tensorflow.keras.regularizers import l2

`BatchNormalization(axis, momentum, epsilon)` : <br>
https://keras.io/api/layers/normalization_layers/batch_normalization/
- axis: Batch normalization이 적용될 axis. 우리는 채널에 대해서 BN을 적용할 것이다.
- momentum: Moving average에 적용될 momentum 계수
- epsilon: 0으로 나누는 것을 방지하기 위한 작은 수.


`DepthwiseConv2D(kernel_size, strides, padding, use_bias, depthwise_regularizer)` : https://keras.io/api/layers/convolution_layers/depthwise_convolution2d/

## GPU Setting

In [25]:
#os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [27]:
"""
Make sure your runtime type is GPU!
"""
physical_devices = tf.config.list_physical_devices('GPU')
print('Num_GPUs:{}, List:{}'.format(len(physical_devices), physical_devices))

Num_GPUs:0, List:[]


In [6]:
gpu_growth = False

if gpu_growth:
    physical_devices = tf.config.list_physical_devices('GPU')
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass

---

## 0. Dataset : Cifar-10

### Load Dataset

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

170498071/170498071 [==============================] - 2s 0us/step


### Data Visualization

In [8]:
i = np.random.randint(len(x_train))
names = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}
plt.imshow(x_train[i])
plt.title(names[y_train[i][0]])
plt.show()

In [9]:
# 50,000장 중 5,000장 이용
indices = np.random.randint(low=0, high=len(x_train), size=5000)
x_train, y_train = x_train[indices], y_train[indices]

---

## 1. MobileNetV2

paper: [MobileNetV2](https://openaccess.thecvf.com/content_cvpr_2018/papers/Sandler_MobileNetV2_Inverted_Residuals_CVPR_2018_paper.pdf)

### (1) Inverted Residual and Linear Bottleneck


<img src="https://user-images.githubusercontent.com/37704174/112773048-5bbfd400-906f-11eb-8e0c-62558ba0a1a2.PNG" width="500" height="500"/>

<img src="https://user-images.githubusercontent.com/37704174/112772900-5e6df980-906e-11eb-9c02-5310fa51ff43.PNG" width="300" height="300"/>

inverted_res.png

#### Expansion
  - `expansion != 1`인 <b>경우에만</b> 1x1 convolution을 이용하여 channel의 수를 `expansion`배 만큼 증가시킴
  - `strides=1`, `padding='same'`, `use_bias=False`, `kernel_regularizer=l2(4e-5)`
  - `BatchNormalization`과 ReLU6을 이용
    - BN의 `momentum=0.999`, `epsilon=0.001`
    - ReLU6은 ReLU함수에서 최대값이 6인 함수
    
#### Depthwise convolution
  - 3x3 `DepthwiseConv2D` layer를 이용
  - `strides`를 함수 입력으로 받음,  `padding=same`, `use_bias=False`, `depthwise_regularizer=l2(4e-5)`
  - `BatchNormalization`과 ReLU6을 이용
    - 위와 같음
    
    
   
#### Linear bottleneck
  - 1x1 convolution으로 channel의 수를 함수 입력으로 받는 `filters`로 줄임
  - `strides=1`, `padding='same'`, `use_bias=False`, `kernel_regularizer=l2(4e-5)`
  - `BatchNormalization`을 이용
    - 위와 같음
  - 'Linear' bottleneck이기 때문에 Activation은 이용하지 않음


#### Residual
  - input channel의 수와 `filters`가 같고, `strides`가 1인 경우에 원래 들어왔던 inputs와 Linear bottleneck의 output을 `Add` 해준다



In [16]:
def _inverted_res_block(inputs, expansion, filters, strides):
    x = inputs
    in_chnls = inputs.shape[-1]

    ########################## 실습 ##########################
    # Expansion
    if expansion != 1:
      x = Conv2D(kernel_size=1,filters=in_chnls * expansion, strides=1, padding='same', use_bias=False, kernel_regularizer=l2(4e-5))(x)
      x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
      x = ReLU(max_value=6)(x)

    # Depthwise convolution
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same', use_bias=False, depthwise_regularizer=l2(4e-5))(x)
    x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)

    # Linear bottleneck
    x = Conv2D(kernel_size=1, filters=filters, strides=1, padding='same', use_bias=False, kernel_regularizer=l2(4e-5))(x)
    x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)

    # Residual connection
    if in_chnls == filters and strides == 1:
      x = Add()([inputs, x])

    #########################################################

    return x #return output of layer

### (2) Mobilenet V2 Network
위에서 만든 `_inverted_res_block`을 이용하여 MobilenetV2를 구성해 보도록 하자. Network의 구조는 아래 표에 요약되어 있다. <br>
<img src="https://user-images.githubusercontent.com/37704174/112775798-e527d400-9078-11eb-93b1-33cdf2f3e6f7.PNG" width="500" height="500"/>  

- `_inverted_res_block`에서 개수가 1보다 큰 것들은 첫번재 block만 `strides`가 2이고, 나머지는 1임
- Conv2D layer 다음에는 BN과 ReLU6을 이용 (`_inverted_res_block`과 hyperparameter는 같게 설정)
- Dense layer는 `activation='softmax'` 이용

In [17]:
def MobileNetV2(input_shape, classes):
    inputs = Input(shape=input_shape)

    x = Conv2D(filters=32, kernel_size=3, strides=2, padding='same')(inputs)
    x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)

    ########################## 실습 ##########################
    # inverted residual blocks
    x = _inverted_res_block(inputs=x, expansion=1, filters=16, strides=1)

    x = _inverted_res_block(inputs=x, expansion=6, filters=24, strides=2)
    x = _inverted_res_block(inputs=x, expansion=6, filters=24, strides=1)

    x = _inverted_res_block(inputs=x, expansion=6, filters=32, strides=2)
    x = _inverted_res_block(inputs=x, expansion=6, filters=32, strides=1)
    x = _inverted_res_block(inputs=x, expansion=6, filters=32, strides=1)

    x = _inverted_res_block(inputs=x, expansion=6, filters=64, strides=2)
    x = _inverted_res_block(inputs=x, expansion=6, filters=64, strides=1)
    x = _inverted_res_block(inputs=x, expansion=6, filters=64, strides=1)
    x = _inverted_res_block(inputs=x, expansion=6, filters=64, strides=1)

    x = _inverted_res_block(inputs=x, expansion=6, filters=96, strides=1)
    x = _inverted_res_block(inputs=x, expansion=6, filters=96, strides=1)
    x = _inverted_res_block(inputs=x, expansion=6, filters=96, strides=1)

    x = _inverted_res_block(inputs=x, expansion=6, filters=160, strides=2)
    x = _inverted_res_block(inputs=x, expansion=6, filters=160, strides=1)
    x = _inverted_res_block(inputs=x, expansion=6, filters=160, strides=1)

    x = _inverted_res_block(inputs=x, expansion=6, filters=320, strides=1)

    #########################################################

    x = Conv2D(kernel_size=1, filters=1280, strides=1, padding='same')(x)
    x = BatchNormalization(momentum=0.999, epsilon=0.001)(x)
    x = ReLU(max_value=6)(x)

    # global average pooling
    x = GlobalAveragePooling2D()(x)

    # FC layer
    outputs = Dense(classes, activation='softmax')(x)

    return Model(inputs=inputs, outputs=outputs)

In [18]:
model = MobileNetV2(input_shape=(32,32,3), classes=10)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 16, 16, 32)   896         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_3 (BatchNo  (None, 16, 16, 32)  128         ['conv2d_2[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 re_lu_2 (ReLU)                 (None, 16, 16, 32)   0           ['batch_normalization_3[0][0]

---

## 2. Training

In [19]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [20]:
model.fit(x=x_train, y=y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test), shuffle=True)

Epoch 1/50
157/157 [==============================] - 59s 287ms/step - loss: 2.7156 - accuracy: 0.1464 - val_loss: 2.4872 - val_accuracy: 0.1000
Epoch 2/50
157/157 [==============================] - 46s 297ms/step - loss: 2.3018 - accuracy: 0.2294 - val_loss: 2.4915 - val_accuracy: 0.1000
Epoch 3/50
157/157 [==============================] - 46s 295ms/step - loss: 2.1238 - accuracy: 0.2844 - val_loss: 2.5000 - val_accuracy: 0.1000
Epoch 4/50
  7/157 [>.............................] - ETA: 53s - loss: 1.9922 - accuracy: 0.3482

KeyboardInterrupt: ignored